In [0]:
!pip install python_speech_features 
import os
import sys
import wave
import timeit; program_start_time = timeit.default_timer()
import random; random.seed(int(timeit.default_timer()))
import numpy as np
import scipy.io.wavfile as wav
import python_speech_features as features
import h5py
from sklearn.model_selection import train_test_split
def create_mfcc(filename):
	"""
	Output consists of 12 MFCC and 1 energy, as well as the first derivative of these.
	[1 energy, 12 MFCC, 1 diff(energy), 12 diff(MFCC)
	"""
	#print filename
	(rate,sample) = wav.read(filename)

	mfcc = features.mfcc(sample, rate, winlen=0.025, winstep=0.01, numcep = 13, nfilt=26,
	preemph=0.97, appendEnergy=True)
	d_mfcc = features.delta(mfcc, 2)
	a_mfcc = features.delta(d_mfcc, 2)

	out = np.concatenate([mfcc, d_mfcc, a_mfcc], axis=1)

	return out, out.shape[0]
def calc_norm_param(X):
	"""Assumes X to be a list of arrays (of differing sizes)"""
	total_len = 0
	mean_val = np.zeros(X[0].shape[1])
	std_val = np.zeros(X[0].shape[1])
	for obs in X:
		obs_len = obs.shape[0]
		mean_val += np.mean(obs,axis=0)*obs_len
		std_val += np.std(obs, axis=0)*obs_len
		total_len += obs_len
	
	mean_val /= total_len
	std_val /= total_len

	return mean_val, std_val, total_len

In [0]:
def normalize(X, mean_val, std_val):
	for i in range(len(X)):
		X[i] = (X[i] - mean_val)/std_val
	return X
x=[]
y_final=[]
f=open('wav_list','r')
#For train_mfcc
k=0
for i in f.readlines():
 i=i.replace('\n','')#print i
 i_mfcc, total_frames = create_mfcc(str(i))
 x.append(i_mfcc)
 k=k+1
 print(k)
tgt_dict={"one":1,"zero":0,"two":2,"three":3,"four":4,"five":5,"six":6,"seven":7,"eight":8,"nine":9}
f.close()
k=0
f=open('target_list','r')
for i in f.readlines():
        print(i)
        i=i.replace('\n','')
        print(tgt_dict[i])
        y_final.append(tgt_dict[i])
        k=k+1
        print(k)
mean,std,length=calc_norm_param(x)
x_final=normalize(x,mean,std)
f.close()
x_train,x_temp,y_train,y_temp=train_test_split(x_final, y_final, test_size=0.3, random_state=1)
x_test,x_dev,y_test,y_dev=train_test_split(x_temp, y_temp, test_size=0.5, random_state=1)

In [0]:
f=h5py.File("feats_labels_new_train.h5",'w')
for i in range(0,len(y_train)):
 d=x_train[i]
 s=np.repeat(int(y_train[i]),len(d))
 a=np.reshape(s,(len(s),1))
 dd=np.hstack([d,a])
 f.create_dataset(str(i),data=dd)
f.close()

f=h5py.File("feats_labels_new_test.h5",'w')
for i in range(0,len(y_test)):
 d=x_test[i]
 s=np.repeat(int(y_test[i]),len(d))
 a=np.reshape(s,(len(s),1))
 dd=np.hstack([d,a])
 f.create_dataset(str(i),data=dd)
f.close()

f=h5py.File("feats_labels_new_dev.h5",'w')
for i in range(0,len(y_dev)):
 d=x_dev[i]
 s=np.repeat(int(y_dev[i]),len(d))
 a=np.reshape(s,(len(s),1))
 dd=np.hstack([d,a])
 f.create_dataset(str(i),data=dd)
f.close()